In [1]:
import torch
from torch import nn, optim
import time
import torchvision

import sys
sys.path.append('..')
import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )
        
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

#### 读取数据&resize

In [3]:
def load_data_fashion_mnist(batch_size, resize=None, root=None):
    '''
    获取并读取数据
    root：数据存储位置
    如果数据已经下载过，将download=False
    '''
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=False, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=False, transform=transform)
    
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_iter, test_iter

#### 训练

In [4]:
batch_size = 64
data_dir = './Datasets/FashionMNIST'
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224, root=data_dir)

In [6]:
net = AlexNet()

In [7]:
lr, num_epochs = 0.001, 5
optimizer = optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.5417, train_acc 0.796, test acc 0.856, 80.4 sec
epoch 2, loss 0.1664, train_acc 0.876, test acc 0.895, 77.7 sec
epoch 3, loss 0.0964, train_acc 0.894, test acc 0.894, 78.5 sec
epoch 4, loss 0.0652, train_acc 0.906, test acc 0.899, 77.7 sec
epoch 5, loss 0.0479, train_acc 0.909, test acc 0.909, 77.5 sec
